In [31]:

def write_wcnf_file(weights, clauses, file_path):
    with open(file_path, 'w') as file:
        # Write the weights and clauses
        for weight, clause in zip(weights, clauses):
            clause_line = ' '.join(map(str, clause))
            file.write(f'{weight} {clause_line} 0\n')

weights = [0.9 ,0.9 ,0.78,0.65,0.65,0.65,0.6,0.6,0.58,0.43,0.43,0.43,0.36,0.3,0.26,0.14]
clauses = [
    [1, -2, 3, 4],
    [-2, 3],
    [2, -3],
    [1, -2, 3, 4],
    [-2, 3],
    [-1, 2, -3],
    [1, -2, 3, 4],
    [-2, 3],
    [-1, -2, 3],
    [1, -2, 3, 4],
    [-2, 3],
    [-1, 2, -3],
    [1, 5],
    [-2, 3, 6],
    [-1, 4],
    [4],
    [2, -3],
]


write_wcnf_file(weights, clauses, 'file.wcnf')


## 1) Generation de la base de connaissance

In [32]:
import random
import os

class Sigma:
    _lower = 0
    _upper = 0
    _weights = []
    _formulas = []
    strates_weights = []
    def __init__(self,path_file): 
        with open(path_file, 'r') as file:
            lines = file.readlines()

        for line in lines:
            information = line.split()
            size = len(information)
            self._weights.append(float(information[0]))
            self._formulas.append([int(information[i]) for i in range(size) if i != 0]) 

        self._length = len(self.weights)
        file.close()

    @property
    def length(self):
        return self._length
    
    @property
    def weights(self):
        return self._weights
    
    @property
    def formulas(self):
        return self._formulas
    
    @property    
    def upper(self):
        return self._upper
    
    @property
    def lower(self):
        return self._lower
    
    @weights.setter
    def weights(self, value):
        self._weights = value

    @formulas.setter
    def formulas(self, value):
        self._formulas = value
    
    @upper.setter
    def upper(self, value):
        self._upper = value

    @lower.setter
    def lower(self, value):
        self._lower = value

    def sort_weights(self):
        """ Sort the weights and formulas in the same time """
        self.weights, self.formulas = zip(*sorted(zip(self.weights, self.formulas), reverse=True))
        self.weights = list(self.weights)
        self.formulas = list(self.formulas)

    def compute_strates(self): 
        no_doubles = set(self.weights)
        self._upper = len(no_doubles)
        self.strates_weights = sorted(no_doubles)

    def get_strates_weights(self):
        return self.strates_weights

    def get_preprocessed_formulas(self,sub_formulas):
        dict_cor={}
        returned_formulas=[]
        cpt=1
        i=0
        for form in sub_formulas:
            j=0
            returned_formulas.insert(i,[])
            for pred in form:
                if(pred not in dict_cor.keys()):
                    if(int(pred)>0):
                        dict_cor[int(pred)]=cpt
                        dict_cor[-int(pred)]=-cpt
                        cpt+=1
                    else:
                        dict_cor[-int(pred)]=cpt
                        dict_cor[int(pred)]=-cpt
                        cpt+=1
                returned_formulas[i].insert(j,dict_cor[pred])
                j+=1
            i+=1
        return returned_formulas
    



In [33]:
class Generate:
    number_of_clause   = 0
    number_of_variable = 0
    evidence_length = 0
    var_list = []
    max_clause_length = 0

    def __init__(self,nb_clause,nb_variable,length,max_number):
        self.number_of_clause = nb_clause
        self.number_of_variable = nb_variable
        self.evidence_length = length
        self.max_clause_length = max_number
        for i in range(self.number_of_variable):
            self.var_list.append(i+1)
            self.var_list.append(-(i+1))

    def get_evidence(self):
        return random.sample(self.var_list, self.evidence_length)

    def get_sigma(self):
        os.system("touch temp.txt")
        sigma_file = open("temp.txt","w")
        for i in range(self.number_of_clause):
            ran = random.randrange(1,self.max_clause_length + 1,1)
            temp_list = random.sample(self.var_list, ran)
            weight = random.random()
            weight = float(int(weight*100))/100
            chaine = "" + str(weight)+ " "
            for i in range(len(temp_list)):
                chaine = chaine + str(temp_list[i]) + " "
            sigma_file.write(chaine + "\n")
            temp_list=[]
        sigma_file.close()

In [34]:
from pysat.solvers import Glucose3

choice = 1
if choice == 1:
    sigma = Sigma("file.wcnf")
    evidence = [1, -2, -3]
else:
    nb_clause = 8
    nb_variable = 5
    length = 3
    max_number = 4
    gen = Generate(nb_clause, nb_variable, length, max_number)
    gen.get_sigma()
    sigma = Sigma("temp.txt")
    evidence = gen.get_evidence()

# Sort weights and calculate strates using Sigma
sigma.sort_weights()
sigma.compute_strates()

# Continue with the rest of your code using sigma instead of base_de_connaissance
iteration = 1

while sigma.lower < sigma.upper:
    r = int((sigma.lower + sigma.upper + 1) / 2)

    liste = sigma.weights
    value_of_r = -1

    for i in range(len(liste)):
        if sigma.get_strates_weights()[r - 1] > liste[i]:
            value_of_r = i
            break

    if value_of_r == -1:
        value_of_r = len(liste) - 1

    for j in range(len(liste)):
        if sigma.get_strates_weights()[sigma.upper - 1] == liste[j]:
            valueU = j
            break

    cnf = sigma.formulas[valueU:value_of_r]

    for i in range(len(evidence)):
        cnf.append([-1 * evidence[i]])

    cnf = sigma.get_preprocessed_formulas(cnf)
    solver = Glucose3()
    solver.append_formula(cnf)

    if solver.solve():
        sigma.upper = r - 1
    else:
        sigma.lower = r

    iteration += 1

Val = sigma.get_strates_weights()[r - 1]

print("L'exécution d'une base de connaissance qui contient : ")
for clause in sigma.formulas:
    print(clause)
print("Et avec une évidence de : ", evidence)
print("Donne le résultat suivant : Val(" + str(evidence) + ", Sigma) = " + str(Val))


L'exécution d'une base de connaissance qui contient : 
[1, -2, 3, 4, 0]
[-2, 3, 0]
[2, -3, 0]
[1, -2, 3, 4, 0]
[-1, 2, -3, 0]
[-2, 3, 0]
[1, -2, 3, 4, 0]
[-2, 3, 0]
[-1, -2, 3, 0]
[1, -2, 3, 4, 0]
[-1, 2, -3, 0]
[-2, 3, 0]
[1, 5, 0]
[-2, 3, 6, 0]
[-1, 4, 0]
[4, 0]
Et avec une évidence de :  [1, -2, -3]
Donne le résultat suivant : Val([1, -2, -3], Sigma) = 0.14
